In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\\Windows\\Fonts\\malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import bs4
import requests

import json
import re
from tqdm import notebook

In [2]:
# json 파일 생성
data = {}
data["data"] = []

with open('recipe_data.json', 'w', encoding="utf-8") as make_file:    
    json.dump(data, make_file, ensure_ascii=False, indent="\t")

In [ ]:
header = {"User-Agent" : "Mozilla/5.0"}

# 해당 사이트에 총 25개의 카테고리가 있음을 확인(1 ~ 25)
for i in notebook.tqdm(range(25)): # 0 ~ 24
    theme_url = "https://2bob.co.kr/recipe.php?id=list&fKeyList=&fKeyValue=&eTheme={}"
    theme_url = theme_url.format(i+1)


    # 한개의카테고리에 몇개의 큰 페이지(5개 묶음 ex> 1 ~ 5, 6 ~ 10, 11 ~ 12)가 있는지 파악
    # ex) 3페이지일 경우 리스트가 [1, 6, 11]
    # ex) 2페이지일 경우 리스트가 [1. 6]
    next_page_num_list = [1]
    k = 0
    while(True):
        # 웹 소스 준비
        url = theme_url + "&OrderCondition=&OrderBy=&page={}"      
        url = url.format(next_page_num_list[k])
        page = requests.get(url, headers=header)

        soup = BeautifulSoup(page.text, "html.parser")

        if(soup.find("img", alt="마지막으로") != None):    
            # 페이지에서 >> 버튼("마지막으로")이 있을경우 다음 큰 페이지가 있음을 의미함
            next_Page = soup.find("img", alt="마지막으로").parent
        else:
            break   # 마지막으로 버튼이 없을 경우 해당 카테고리의 마지막 큰 페이지이므로 반복문 종료
        if(next_page_num_list[k] < int(next_Page["href"].split("=")[-1])):
            next_page_num_list.append(int(next_Page["href"].split("=")[-1]))
            k = k + 1
#             print(next_Page["href"].split("=")[-1])
        else:
            break

#     print(next_page_num_list)

    for lead_page_num in notebook.tqdm(next_page_num_list):

        # 웹 소스 준비
        url = theme_url + "&OrderCondition=&OrderBy=&page={}"     
        url = url.format(lead_page_num)       
        page = requests.get(url, headers=header)

        # 가져온 웹 소스를 트리 구조로 변환
        soup = BeautifulSoup(page.text, "html.parser")

        # urljoin을 위한 base 선언
        base = "https://2bob.co.kr"
        base2 = "https://2bob.co.kr/recipe.php"

        # 한개의 카테고리의 페이지 url 가져오기
        page_list = []
        page_list.append(url)

        if(soup.find("p", "paging_num") != None):
            if(soup.find("p", "paging_num").a != None):
                page_list.append(soup.find("p", "paging_num").a["href"])
                next_pages = soup.find("p", "paging_num").a.next_siblings

                for page in next_pages:
                    page_list.append(page["href"])

        for page in notebook.tqdm(page_list):
            if(page == url):
                url = page
            else:
                url = base2 + page

            page = requests.get(url, headers=header)

            # 가져온 웹 소스를 트리 구조로 변환
            soup = BeautifulSoup(page.text, "html.parser")

            # 페이지당 4줄, 4줄의 맨 앞에있는 레시피만 ml_none 클래스 부여되어있음
            # 따라서 뒤에 따라오는 4개의 레시피는 해당 레시피 기준 siblings을 사용하여 가져옴

            # 가로기준 맨 앞 항목의 링크 가져오기
            urls = soup.select('li.ml_none > a')
            lead_urls_list = []
            for url in urls:
                lead_urls_list.append(url["href"])

            # 맨 앞 항목의 링크를 기준으로 따라오는 4개의 링크 가져오기
            follow_url_list=[]
            for lead_url in lead_urls_list:
                urls = soup.find("a", href=lead_url).parent.next_siblings
                for url in urls:
                    if(type(url) == bs4.element.Tag):
                        follow_url_list.append(list(url.children)[1]["href"])

            # 두개의 리스트 합치기
            url_list = lead_urls_list + follow_url_list

            # 합쳐진 리스트의 중복 제거
            url_list = set(url_list)
            url_list = list(url_list)

            # 괄호, Tip, \r\n 제거를 위한 패턴 생성
            pattern0 = r'\([^)]*\)'
            pattern1 = r'\([^)]*$'
            pattern2 = 'TIP.*?요.'
            pattern3 = 'tip.*?요.'
            pattern4 = '\r\n'
            pattern5 = '\n'

            # 생성해 놓은 json파일 load
            with open('recipe_data.json', "r", encoding='utf-8') as json_file:    
                json_data = json.load(json_file)

            if(len(json_data["data"]) == 0): # 처음 저장할 경우
                cnt = 0
            else: # 기존에 테이터가 있을 경우
                cnt = json_data["data"][-1]["code"] + 1   

            for i in notebook.tqdm(url_list):
                # 웹 소스 준비
                url = urllib.parse.urljoin(base, i)

                header = {"User-Agent" : "Mozilla/5.0"}
                page = requests.get(url, headers=header)
                page
                # 가져온 웹 소스를 트리 구조로 변환
                soup = BeautifulSoup(page.text, "html.parser")

                # 레시피 명
                if (soup.find('h2', class_="prod_title") != None):
                    title = (soup.find('h2', class_="prod_title").get_text()).strip()
                else :
                    title = "???"
#                print(title)

                # 필수재료(좌우 공백제거)
                if (soup.find('p', class_="mate_list") != None):
                    ingredients = soup.find('p', class_="mate_list").get_text()
                else :
                    ingredients = "???"

                # 재료명에서 괄호와 괄호안의 정보 삭제
                ingredients = re.sub(pattern=pattern0, repl='', string=ingredients)
                ingredients = re.sub(pattern=pattern1, repl='', string=ingredients)
                
                # 재료명에서 TIP/tip 문장 제거
                ingredients = re.sub(pattern=pattern2, repl='', string=ingredients)
                ingredients = re.sub(pattern=pattern3, repl='', string=ingredients)

                # 재료명에서 \r\n 제거
                ingredients = re.sub(pattern=pattern4, repl='', string=ingredients)
                
                # 재료명에서 \n 제거
                ingredients = re.sub(pattern=pattern5, repl='', string=ingredients)

                # 리스트로 변환
                temp_list = ingredients.split(',')
                
                ingredients_list = []
                # 공백 제거 및 전처리
                for i in temp_list:
                    i = re.sub(r"\s", "", string=i)
                    i = re.sub(r'[\d½⅓][^)]*\)', "", string=i)
                    
                    if ("또는" in i):
                        ingredients_list += (i.split("또는"))
                    elif ("." in i):
                        ingredients_list += (i.split("."))
                    else :                                    
                        if i != '':
                            ingredients_list.append(i)


                # 필수재료 이외의 재료와 양념이 있다면 가져오기
                if(soup.find('p', class_="mate_list").parent.next_siblings != None):
                    siblings = soup.find('p', class_="mate_list").parent.next_siblings
                    for sibling in siblings:
                        if ((type(sibling) == bs4.element.Tag) and (sibling.p != None)):
                            ingredients = sibling.p.get_text()

                            # 재료명에서 괄호와 괄호안의 정보 삭제
                            ingredients = re.sub(pattern=pattern0, repl='', string=ingredients)
                            ingredients = re.sub(pattern=pattern1, repl='', string=ingredients)
                
                            # 재료명에서 TIP/tip 제거
                            ingredients = re.sub(pattern=pattern2, repl='', string=ingredients)
                            ingredients = re.sub(pattern=pattern3, repl='', string=ingredients)

                            # 재료명에서 \r\n 제거
                            ingredients = re.sub(pattern=pattern4, repl='', string=ingredients)

                            # 재료명에서 \n 제거
                            ingredients = re.sub(pattern=pattern5, repl='', string=ingredients)

                            # 리스트로 변환
                            if ("," in ingredients):
                                temp_list = ingredients.split(',')
                            elif ("+" in ingredients):
                                temp_list = ingredients.split('+')
                                    
                            # 공백 제거 및 전처리
                            for i in temp_list:
                                i = re.sub(r"\s", "", string=i)
                                i = re.sub(r'[\d½⅓][^)]*\)', "", string=i)
                                                                        
                                if ("또는" in i):
                                    ingredients_list += (i.split("또는"))
                                elif ("." in i):
                                    ingredients_list += (i.split("."))
                                else :                                    
                                    if i != '':
                                        ingredients_list.append(i)
    
    
#                print(ingredients_list)
                json_data['data'].append({
                    "code" : cnt,
                    "title" : title,
                    "ingredients" : ingredients_list,
                    "url" : url
                })
                 print("--------------------------------------------------")
                cnt = cnt + 1
                # json 파일 저장
                with open('recipe_data.json', "w", encoding='utf-8') as outfile:      ####################################### 수정해야할 곳 ####################
                    json.dump(json_data, outfile, indent="\t", ensure_ascii=False)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

['대파', '맛살', '달걀', '밥', '소금', '맛술', '다진마늘', '설탕', '멸치액젓', '간장', '후춧가루']
['밥', '달걀', '생연어', '오이', '단무지', '김', '밥', '달걀', '생연어', '오이', '단무지', '김', '소금', '설탕', '식초', '소금', '맛술']
['양파', '빨간파프리카', '베이컨', '밥', '김밥용김', '슬라이스모차렐라치즈', '슬라이스체다치즈', '버터', '케첩', '굴소스', '올리고당']
['대파', '양파', '청양고추', '다진쇠고기', '밥', '생크림', '우유', '피자치즈', '설탕', '간장', '다진마늘', '다진생강', '참기름', '맛술', '후춧가루', '소금', '후춧가루']
['현미', '단호박', '견과류', '닭안심', '현미', '단호박', '견과류', '닭안심']
['양파', '대파', '통조림참치', '밥', '양파', '대파', '통조림참치', '밥', '간장', '마요네즈', '후춧가루', '참깨']
['통조림참치', '묵은지', '밥', '소금', '참기름', '참깨', '마요네즈']
['프랑크소시지', '슬라이스체더치즈', '파프리카', '김밥용김', '밥', '참기름', '청상추', '프랑크소시지', '슬라이스체더치즈', '파프리카', '김밥용김', '밥', '참기름', '청상추']
['베이컨', '양파', '아보카도', '콜리플라워라이스', '다진마늘', '후춧가루', '레몬즙', '간장', '꿀', '레몬제스트']
['당근', '시금치', '햄', '맛살', '달걀물', '밥', '김', '단무지', '당근', '시금치', '햄', '맛살', '달걀물', '밥', '김', '단무지', '소금', '참기름', '참깨']
['당근', '피망', '통단무지', '돈가스', '밥', '김', '깻잎', '소금', '참기름', '돈가스소스']
['당근', '시금치', '참치', '묵은지', '달걀', '밥', '김', '우엉조림', '단무지',

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

['애호박', '감자', '두부', '돼지고기', '양파', '청양고추', '홍고추', '국물용멸치', '다시마', '소금', '다진마늘', '후춧가루', '고추장', '된장']
['애호박', '감자', '양파', '풋고추', '국물용멸치', '두부', '청양고추', '대파', '된장', '고추장', '고춧가루', '다진마늘']
['느타리버섯', '콩나물', '미나리', '알배추', '당근', '손질한주꾸미', '무', '양파', '대파', '다시마', '국물용멸치', '무', '양파', '대파', '다시마', '국물용멸치', '간장', '식초', '연겨자', '레몬청', '물']
['양배추', '표고버섯', '우엉', '닭가슴살', '우유', '슬라이스체다치즈', '당근', '양파', '소금', '후춧가루', '밀가루', '국간장']
['김치', '대파', '김칫국물', '어묵국물', '꼬치어묵', '우동사리', '김치', '대파', '김칫국물', '어묵국물', '꼬치어묵', '우동사리', '김치', '대파', '김칫국물', '어묵국물', '꼬치어묵', '우동사리']
['알배추', '깻잎', '숙주', '표고버섯', '팽이버섯', '쑥갓', '황태채', '다시마육수', '들기름', '새우젓']
['당면', '양파', '부추', '팽이버섯', '흑돼지앞다릿살', '설탕', '고춧가루', '후춧가루', '간장', '매실액', '다진마늘', '다진생강', '고추장', '참기름']
['두부', '어묵', '청양고추', '두부', '어묵', '청양고추', '설탕', '소금', '고춧가루', '새우젓', '다진마늘']
['양파', '애호박', '감자', '소고기', '쌀뜨물', '대파', '청양고추', '홍고추', '참기름', '다진마늘', '고추장', '된장']
['냉이', '달래', '두부', '애호박', '양파', '멸치', '다시마', '참기름', '다진마늘', '다진파', '고춧가루', '된장']
['숙주', '묵은지', '쇠고기샤브샤브용', '깻잎', '청정

  0%|          | 0/20 [00:00<?, ?it/s]

['총알오징어', '양송이버섯', '표고버섯', '새송이버섯', '느타리버섯', '팽이버섯', '대파', '양파', '총알오징어', '양송이버섯', '표고버섯', '새송이버섯', '느타리버섯', '팽이버섯', '대파', '양파', '설탕', '고춧가루', '국간장', '다진마늘', '된장', '고추장', '따뜻한물', '다시마Tip_따뜻한물에다시마를30분동안담가주세요', '']
['열무', '대파', '청양고추', '돼지고기', '멸치', '다시마', '고춧가루', '청국장', '된장']
['양파', '두부', '무', '대파', '우삼겹', '애호박청양고추', '홍고추', '쌀뜨물', '고춧가루', '된장', '고추장', '다진마늘']
['애호박', '감자', '양파', '대파', '새송이버섯', '붉은고추', '국물용멸치', '다시마', '고추장', '된장', '설탕', '다진마늘']
['국물용멸치', '다시마', '가지', '감자', '양파', '풋고추', '대파', '고추장', '국간장', '소금', '다진마늘', '후춧가루']
['두부', '양파', '파김치', '손질된장어', '청양고추', '홍고추', '쑥갓', '다시마', '물', '생강즙', '멸치액젓', '다진마늘']
['굴', '쇠고기불고기용', '당근', '배추', '무', '양파', '불린표고버섯', '붉은고추', '풋고추', '팽이버섯', '불린당면', '간장', '다진파', '다진마늘', '후춧가루', '통깨', '설탕', '참기름', '진육수', '소금']
['바지락', '대파', '양파', '순두부', '애호박', '다시마', '된장', '고춧가루', '다진마늘']
['무', '콩나물', '게', '낙지', '중하', '미더덕', '미나리', '배추', '전복', '가리비', '고춧가루', '청주', '국간장', '다진생강', '다진마늘', '고추장', '된장']
['만두피', '두부', '양파', '느타리버섯', '표고버섯', '대파', '풋고추', '붉은고추', '미나리', '